In [7]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import random

class CarAgent(Agent):
    """ An agent representing a car looking for a parking space. """
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.parked = False
        self.parked_steps = 0

    def move(self):
        """ Move to a new, random position. """
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def find_parking(self):
        """ Find a parking space and park if available. """
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        for mate in cellmates:
            if isinstance(mate, ParkingSpaceAgent) and not mate.occupied:
                mate.occupied = True
                self.parked = True
                self.parked_steps = random.randint(3, 5)
                break

    def step(self):
        if not self.parked:
            self.move()
            self.find_parking()
        else:
            if self.parked_steps > 0:
                self.parked_steps -= 1
            else:
                self.parked = False
                cellmates = self.model.grid.get_cell_list_contents([self.pos])
                for mate in cellmates:
                    if isinstance(mate, ParkingSpaceAgent):
                        mate.occupied = False

class ParkingSpaceAgent(Agent):
    """ An agent representing a parking space. """
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.occupied = False

    def step(self):
        pass  # Parking spaces do not have a step method

class ParkingLotModel(Model):
    """ A model representing a parking lot with multiple cars and parking spaces. """
    def __init__(self, N, width, height):
        self.num_agents = N
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)

        # Create parking spaces
        for i in range(5):
            parking_space = ParkingSpaceAgent(i, self)
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(parking_space, (x, y))
            self.schedule.add(parking_space)

        # Create cars
        for i in range(N):
            car = CarAgent(i + 100, self)
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(car, (x, y))
            self.schedule.add(car)

        self.datacollector = DataCollector(
            agent_reporters={"Position": "pos", "Parked": "parked"}
        )

    def step(self):
        """ Advance the model by one step. """
        self.datacollector.collect(self)
        self.schedule.step()


In [8]:
import matplotlib.pyplot as plt
import numpy as np

def draw_model_state(model):
    # Define colors
    road_color = [255, 255, 255]  # White
    car_color = [0, 0, 255]       # Blue
    parking_color = [0, 255, 0]   # Green
    occupied_color = [255, 0, 0]  # Red

    # Initialize grid
    grid = np.zeros((model.grid.width, model.grid.height, 3), dtype=np.uint8) + road_color

    # Iterate over every cell in the grid
    for x in range(model.grid.width):
        for y in range(model.grid.height):
            contents = model.grid.get_cell_list_contents((x, y))
            contains_car = any(isinstance(agent, CarAgent) for agent in contents)
            contains_space = any(isinstance(agent, ParkingSpaceAgent) for agent in contents)
            is_occupied = contains_car and contains_space

            if is_occupied:
                grid[y, x] = occupied_color
            elif contains_car:
                grid[y, x] = car_color
            elif contains_space:
                grid[y, x] = parking_color

    return grid

# Parameters for the simulation
num_cars = 10
width, height = 10, 10
num_steps = 20

# Create and run the model
model = ParkingLotModel(num_cars, width, height)

for i in range(num_steps):
    model.step()
    grid_image = draw_model_state(model)
    plt.imshow(grid_image, interpolation='nearest')
    plt.axis('off')
    plt.title(f'Step {i}')
    plt.savefig(f'parking_lot_step_{i}.png', bbox_inches='tight')
    plt.close()
